In [1]:
'''
  Install packages required to connect to Clooud MySQL instance
'''
# Install PyMySQL python module
!pip install cloud-sql-python-connector

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.33.0 which is incompatible.


In [1]:
# Install PyMySQL python module
!pip install pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.7 MB/s eta 0:00:00


In [2]:
'''
  Configure Google Cloud Project Context
'''
from tools import load_db_cfg
dbcfg= load_db_cfg('pp.json')
!gcloud config set project {dbcfg['project_id']}

Updated property [core/project].


In [3]:
'''
  Aunthenticate to Google Colab
'''
from google.colab import auth
auth.authenticate_user()

In [4]:
'''
  Install the packages used by the webapp
'''
# Streamlit framework
!pip install -q streamlit

# Streamlit Options Menu - for Left sidebar
!pip install streamlit-option-menu




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.4/823.4 kB 34.3 MB/s eta 0:00:00


In [5]:
# Streamlit Top Navigation Menu - for Left sidebar
!pip install streamlit-navigation-bar

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 3.6 MB/s eta 0:00:00


In [6]:
!pip install folium

In [7]:
!pip install streamlit-folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.0/327.0 kB 7.0 MB/s eta 0:00:00


In [8]:
%%writefile phonepe.py
'''
  Phonepe - Streamlit Webapp
'''
import streamlit as st
from streamlit_option_menu import option_menu
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import traceback
import locale
import plotly.express as px
import plotly
import json
import requests
from datetime import date

sys.path.append(str(Path('pp_dao.py').resolve()))
from pp_dao import PhonePeDao
from tools import load_db_cfg


# Load the DB config and initialize the DAO
dao = PhonePeDao(load_db_cfg('pp.json'))

# Initialize session state variables
if 'supported_years' not in st.session_state:
  st.session_state.supported_years=dao.supported_years()

if 'supported_quarters' not in st.session_state:
  st.session_state.supported_quarters=dao.supported_quarters()

if 'supported_states' not in st.session_state:
  st.session_state.supported_states=dao.supported_states()

if 'year' not in st.session_state:
  st.session_state.year = None

if 'quarter' not in st.session_state:
  st.session_state.quarter = None

if 'state' not in st.session_state:
  st.session_state.state = None

if 'rec_type' not in st.session_state:
  st.session_state.rec_type='Transaction'

if 'data' not in st.session_state:
  st.session_state.data=dao.get_dashboard(st.session_state.rec_type,st.session_state.year,st.session_state.quarter,st.session_state.state)

def display_quarter(quarter):
  return quarter[1]

def display_state(state):
  #return st.session_state.supported_states.iloc[state,'State'].value
  return st.session_state.supported_states['State'][state-1]

@st.cache_data
def load_geojson(f):
  return json.load(open(f, "r"))

# Geo Map
def cleanup_name(n):
	if n is None:
		return n
	return ' '.join([x.capitalize() for x in n.split('-')])


# Use full width of the page
st.set_page_config(layout='wide')

with open('ppstyle.css') as f:
  st.markdown(f'<style>{f.read()}</style>',unsafe_allow_html=True)

# Title and Dashboard Selector
#title,toolbar = st.columns([40,60])

st.title('PhonePe Data Analysis:')

#with toolbar:
try:
  option_menu(menu_title=None, options=['Transaction','Insurance', 'User'], orientation='horizontal', key='rec_type')
except:
  print('')

#Fetch the dashboard using  DAO layer
st.session_state.data = dao.get_dashboard(st.session_state.rec_type,st.session_state.year,st.session_state.quarter,st.session_state.state)

#View area title
dashboard=st.container()
with dashboard:
  # Dashboard Title
  st.subheader(f''' {st.session_state.rec_type} Dashboard''')

  # Filter Panel
  filters=st.container()
  with filters:
    st.subheader('Filters')
    f_year, f_quarter, f_state,_ = st.columns([20,20,30,30])
    try:
      f_year.selectbox('Year', options=st.session_state.supported_years, key='year', index=0, placeholder='Choose Year')
    except:
      print('')

    try:
      f_quarter.selectbox('Quarter', options=st.session_state.supported_quarters, key='quarter', format_func=display_quarter, placeholder='Choose Quarter' )
    except:
      print('')

    #try:
      #f_state.selectbox('State', options=st.session_state.supported_states, key='state',format_func=display_state, placeholder='Choose State')
    #except:
      #print('')

  # Map
  if 'by_state' in st.session_state.data and st.session_state.data['by_state']['charts'][0]['data'].shape[0] >0:
    df=st.session_state.data['by_state']['charts'][0]['data']
    df['State'] = df['State'].apply(cleanup_name)
    metric_cols = df.drop(columns=['State Id', 'State']).columns.to_list()
    states_india = load_geojson("states_india.geojson")
    BHOPAL_CENTER={"lat":23.2599, "lon":77.4126}
    fig = px.choropleth(df,
                        locations="State",
                        featureidkey="properties.st_nm",
                        geojson=states_india,
                        color=metric_cols[0],
                        hover_name='State',
                        hover_data=metric_cols,
                        color_continuous_scale='Inferno_r',
                        center=BHOPAL_CENTER,
                        fitbounds='locations',
                        scope='asia',
                        title=f"{metric_cols[0]} By State",
                        height=1500)

    fig.update_layout(height=700, width=950, font=dict(family="Arial, Helvetica, sans-serif",size=24,), title_x=0.5)
    fig.update_layout(coloraxis_colorbar_x=-0.25,)
    fig.update_geos(fitbounds='locations', visible=False)
    st.plotly_chart(fig,user_container_width=True)
  else:
    st.write('No Data For Selected Filters')

  # Summary Metrics
  if 'aggregates' in st.session_state.data:
    summary_view = st.container()
    agg = st.session_state.data['aggregates']
    agg_charts = agg['charts']
    with summary_view:
      st.subheader(f'''{agg['name']}''')
      mc1, mc2, _ = st.columns([25,25,50])
      metric1 = mc1.metric(agg_charts[0]['title'], value=agg_charts[0]['value'])
      metric2 = mc2.metric(agg_charts[1]['title'], value=agg_charts[1]['value'])

  if 'aux_aggregates' in st.session_state.data:
    detail_view = st.container()
    aux_aggs= st.session_state.data['aux_aggregates']
    aux_charts=aux_aggs['charts']
    d=dict()
    d['x']=list()
    d['y']=list()
    for i in range(len(aux_charts)):
      d['x'].append(aux_charts[i]['title'])
      d['y'].append(aux_charts[i]['value'])

    df=pd.DataFrame(d)
    with detail_view:
      st.subheader(f'''{aux_aggs['name']}''')
      fig,ax= plt.subplots(figsize=(27,3))
      sns.barplot(data=df,x='x', y='y',ax=ax,width=0.9,hue='x')
      st.pyplot(fig)

  # Top Statistics
  if 'top' in st.session_state.data:
    top_view = st.container()
    top = st.session_state.data['top']
    top_charts=top['charts']
    with top_view:
      # Subheader for top chart section
      st.subheader(f'''{top['name']}''')
      fig, (a1,a2,a3) = plt.subplots(figsize=(27,3), ncols =3)
      sns.barplot(data=top_charts[0]['data'],x=top_charts[0]['data'].columns[2], y=top_charts[0]['data'].columns[0], ax=a1,orient='h', width=0.3)
      a1.set_title(top_charts[0]['title'], fontsize = 15)
      a1.set_ylabel(top_charts[0]['data'].columns[0])
      a1.set_yticklabels(a1.get_yticklabels(), rotation=45, ha="right")

      sns.barplot(data=top_charts[1]['data'], x=top_charts[1]['data'].columns[2], y=top_charts[1]['data'].columns[0], ax=a2, orient='h',width=0.3)
      a2.set_title(top_charts[1]['title'], fontsize = 15)
      a2.set_ylabel(top_charts[1]['data'].columns[0])
      a2.set_yticklabels(a2.get_yticklabels(), rotation=45, ha="right")

      sns.barplot(data=top_charts[2]['data'], x=top_charts[2]['data'].columns[2], y=top_charts[2]['data'].columns[0], ax=a3, orient='h',width=0.3)
      a3.set_title(top_charts[2]['title'], fontsize = 15)
      a3.set_ylabel(top_charts[2]['data'].columns[0])
      a3.set_yticklabels(a3.get_yticklabels(), rotation=45, ha="right")
      st.pyplot(fig)

  # Trend
  if 'trend' in st.session_state.data:
    trend_view = st.container()
    trend=st.session_state.data['trend']
    trend_charts=trend['charts']
    with trend_view:
      st.subheader(f'''{trend['name']}''')
      fig2, (ta1) = plt.subplots(figsize=(27,3), ncols =1)
      sns.lineplot(data=trend_charts[0]['data'], x=trend_charts[0]['data'].columns[0], y=trend_charts[0]['data'].columns[1],ax=ta1)
      ta1.set_title(trend_charts[0]['title'], fontsize = 15)
      ta1.set_xlabel(trend_charts[0]['data'].columns[0])
      ta1.set_yticklabels(ta1.get_xticklabels(), rotation=45, ha="right")
      st.pyplot(fig2)



Writing phonepe.py


In [ ]:
! wget -q -O - ipv4.icanhazip.com

34.16.219.115


In [ ]:
!streamlit run /content/phonepe.py &>/content/logs.txt &
!npx localtunnel --port 8501

your url is: https://nasty-webs-love.loca.lt
